In [10]:
# pour les imports
import numpy as np


# Zayed

In [11]:
class Cards:
    def __init__(self):
        self.ncards = 20  # number of cards
        self.cards = 4*[7, 8, 9, 10, 11]  # cards (?? 11 pour "A")

    def shuffle(self):
        '''Knuth shuffle'''
        np.random.shuffle(self.cards)

    def hands(self):
        '''distribution of cards by the agent :)'''
        self.hand1 = [self.cards[0], self.cards[2]]
        self.hand2 = [self.cards[1], self.cards[3]]
        # self.sumHand1 = self.cards[0]+self.cards[2]  # the sum of the two cards
        # # for agent (agent is the one who shuffles)
        # self.sumHand2 = self.cards[1]+self.cards[3]


In [12]:
class Enemies :
    '''many enemies'''
    def __init__(self):
        self.maxPoints= 10000  # the maximum amount of points that the enemy has
        self.points= [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

    def enemy1(self, hand1): # the enemy hand
        goforpoints = None
        if hand1[0]+hand1[1] >= 18:  # (the mean of all possible hands)
            goforpoints = np.random.choice(self.points[5:])
        else:
            goforpoints = np.random.choice(self.points[:5])
        return goforpoints



In [13]:
# class Greedy :
#     def __init__(self):
#         self.maxPoints= 10000  # the maximum amount of points that the greedy agent has
#         self.points= [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
    
#     def 
    

In [14]:
def mapTo1422(sampled,a14,b22):

    return int(a14*sampled+b22-a14) # the distribution of an arm (a point) ??

In [15]:
class ThompsonSampling :
    def __init__(self,alist,blist):
        self.maxPoints= 10000  # the maximum amount of points that the agent has
        self.points = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
        # we can keep track of this beta =>  the agent learn  ?????
        self.alist = alist
        self.blist = blist

    def chooseArmToGo(self,hand2,enemyResponse): # like an arm
        # sample the distrubution of the arm number enemyResponse
        estimated =np.random.beta(self.alist[enemyResponse-1],self.blist[enemyResponse-1])

        estimatedSumEnemyHand=mapTo1422(estimated,14,22) # according to the agent technique
        if(np.sum(hand2)>=estimatedSumEnemyHand): # if the agent hand is grater the the sampled enamy hand
            return True # go for
        else :
            return False # go for -1
    def updateBeta(self, gowith,enemyResponse,hand1,hand2):
        h2=np.sum(hand2) # agent
        h1=np.sum(hand1)
        if(gowith) and h1>h2 :
            self.alist[enemyResponse-1]+=1 # shift bata to the right
        elif h1<=h2:
            self.blist[enemyResponse-1]+=1 # shift bata to the left




    

In [9]:
# test
enemy=Enemies()
a=b=[1 for i in range(10)]
agent=ThompsonSampling(a,b)
while(enemy.maxPoints>=10 and agent.maxPoints>=10):
    cards = Cards()
    cards.shuffle()
    cards.hands()
    hand1= cards.hand1
    hand2=cards.hand2
    EnemyGoFor=enemy.enemy1(hand1)
    AgentGo=agent.chooseArmToGo(hand2,EnemyGoFor)
    #show cards
    if(AgentGo):
        if(np.sum(hand1)>np.sum(hand2)):
            # agent loses
            agent.maxPoints-=EnemyGoFor
        if(np.sum(hand2)>np.sum(hand1)):
            # agent loses
            enemy.maxPoints-=EnemyGoFor
    else :
        agent.maxPoints-=1
    # update agent strategy
    agent.updateBeta(AgentGo,EnemyGoFor,hand1,hand2)
    print("=========Enemy========")
    print("hand1 ", hand1)
    print("GoFor", EnemyGoFor)
    print("maxPonts= ", enemy.maxPoints)
    print("=========Agent========")
    print("hand2 ", hand2)
    print("GoFor",AgentGo)
    print("maxPonts= ", agent.maxPoints)
if(enemy.maxPoints <10):
    print("Enemy losed")
if(agent.maxPoints <10):
    print("Agent losed")



=========Enemy========
hand1  [9, 7]
GoFor 1
maxPonts=  9999
=========Agent========
hand2  [10, 11]
GoFor True
maxPonts=  10000
=========Enemy========
hand1  [7, 7]
GoFor 1
maxPonts=  9998
=========Agent========
hand2  [9, 7]
GoFor True
maxPonts=  10000
=========Enemy========
hand1  [11, 9]
GoFor 9
maxPonts=  9998
=========Agent========
hand2  [10, 7]
GoFor False
maxPonts=  9999
=========Enemy========
hand1  [8, 11]
GoFor 7
maxPonts=  9998
=========Agent========
hand2  [9, 8]
GoFor True
maxPonts=  9992
=========Enemy========
hand1  [10, 7]
GoFor 5
maxPonts=  9998
=========Agent========
hand2  [9, 7]
GoFor True
maxPonts=  9987
=========Enemy========
hand1  [9, 11]
GoFor 9
maxPonts=  9989
=========Agent========
hand2  [10, 11]
GoFor True
maxPonts=  9987
=========Enemy========
hand1  [9, 10]
GoFor 7
maxPonts=  9989
=========Agent========
hand2  [10, 9]
GoFor True
maxPonts=  9987
=========Enemy========
hand1  [7, 8]
GoFor 5
maxPonts=  9984
=========Agent========
hand2  [9, 9]
GoFor True
ma

# Abdallahi

# Mermoud